<h1>Extracting results</h1>
<p>Here we extract data required to plot results from the data generated by ... </p>

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 
from glob import glob
import json

<h3>collecting data</h3>
<p>The following function renames some columns, recodes some True/False columns using string values, and outputs a subset of columns into a CSV file that can be used for plotting</p>

In [2]:
def extract_core_results(df):
    df["train_size"] = [
        float(json.loads(holdouts_kwargs)["train_size"])
        for holdouts_kwargs in df.holdouts_kwargs
    ]

    df["features_names"] = [
        json.loads(edge_feature.replace("'", "\""))[0] if pd.notna(edge_feature) else feature_name
        for feature_name, edge_feature in zip(
            df.features_names,
            df["('model_parameters', 'edge_features')"]
        )
    ]
    df["evaluation_negative_sampling_method"] = [
    "DANS"
    if use_scale_free_distribution
    else "UNS"
    for use_scale_free_distribution in df.use_scale_free_distribution
    ]
    df = df[
        [
            pd.isna(a) or a == b
            for a, b in zip(
                df["('features_parameters', 'use_scale_free_distribution')"],
                df["('model_parameters', 'use_scale_free_distribution')"],
            )
        ]
    ]
    df["model_negative_examples"] = [
     "{training}".format(training=("DANS" if f else "UNS"))
        for f in df["('features_parameters', 'use_scale_free_distribution')"]
    ]
    columns = ["evaluation_mode", "features_names","evaluation_negative_sampling_method",
               "model_negative_examples", "accuracy",  "balanced_accuracy", 
               "false_discovery_rate","matthews_correlation_coefficient",
           "precision", "recall", "specificity", "f1_score", "auroc", "auprc"]
    return df[columns].copy()

In [3]:
# Set to directory in which the 'experiments' directory was placed 
indirectory = input()

/Users/robinp/data


<h3>STRING Protein-Protein Association data</h3>

In [4]:
df = pd.concat([
    pd.read_csv(
        path,
        index_col=0
    )
    for path in glob(f"{indirectory}/experiments/Edge Prediction/HomoSapiens/holdout_*/*.csv.gz")
]).reset_index(drop=True)

In [5]:
string_results = extract_core_results(df)

In [6]:
string_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_examples,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
2,train,Walklets GloVe,DANS,UNS,0.504310,0.504299,0.496403,0.008798,0.503597,0.610293,0.398304,0.551835,0.504209,0.502719
3,test,Walklets GloVe,DANS,UNS,0.502032,0.502023,0.498269,0.004130,0.501731,0.602818,0.401227,0.547649,0.501541,0.499348
4,train,Degree,DANS,DANS,0.499494,0.499448,0.500232,-0.005322,0.499768,0.988554,0.010341,0.663899,0.487517,0.474030
5,test,Degree,DANS,DANS,0.499014,0.498969,0.500476,-0.009792,0.499524,0.987761,0.010178,0.663505,0.477872,0.469604
6,train,Degree,DANS,DANS,0.470948,0.470928,0.520309,-0.064339,0.479691,0.685000,0.256856,0.564250,0.490493,0.517127


In [7]:
string_results.to_csv("string_results.csv")

<h3>SLI synthetic lethality results</h3>

In [8]:
df = pd.concat([
    pd.read_csv(
        path,
        index_col=0
    )
    for path in glob(f"{indirectory}/experiments/Edge Prediction/(SLDB | HomoSapiens)/holdout_*/*.csv.gz")
]).reset_index(drop=True)

In [9]:
sli_results = extract_core_results(df)

In [10]:
sli_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_examples,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
0,train,First-order LINE,DANS,DANS,0.697383,0.697383,0.279268,0.396993,0.720732,0.644493,0.750273,0.680484,0.760481,0.745744
1,test,First-order LINE,DANS,DANS,0.626023,0.626023,0.328125,0.261524,0.671875,0.492635,0.759411,0.568461,0.676276,0.687520
4,train,DeepWalk SkipGram,UNS,DANS,0.733915,0.733915,0.286991,0.470100,0.713009,0.782988,0.684842,0.746362,0.786881,0.737826
5,test,DeepWalk SkipGram,UNS,DANS,0.575286,0.575286,0.405738,0.153720,0.594262,0.474632,0.675941,0.527753,0.559374,0.592515
8,train,Walklets CBOW,UNS,DANS,0.903217,0.903217,0.048810,0.811031,0.951190,0.850055,0.956379,0.897783,0.975267,0.960448


In [11]:
sli_results.to_csv("sli_results.csv")

<H1>Aggregating Results</H1>
<p>Here, we calculate the mean and standard deviation of the
    balanced accuracy, false discovery rate, matthews correlation coefficient,
    F1 score, AUROC, and AUPRC.</p>

In [12]:
def get_mean_and_std(df):
    """
    df should be one of string_results or sli_results
    """
    # We are just interested in the following seven graph/random walk methods
    graph_methods = {'First-order LINE', 'DeepWalk SkipGram', 'Walklets CBOW', 'HOPE',
                 'Second-order LINE','DeepWalk CBOW',    'Walklets SkipGram'}
    df = df[df['features_names'].isin(graph_methods)]
    # Rename columns for conciseness
    df =  df.rename(columns={"evaluation_negative_sampling_method": "evaluation", 
                   "evaluation_mode": "mode","features_names":"methods"}, errors="raise")
    # Define the columns that we want to calculate mean and std def for
    evaluation_d = {'balanced_accuracy':['mean','std'],
               'false_discovery_rate':['mean','std'], 
               'matthews_correlation_coefficient':['mean','std'], 
               'f1_score':['mean','std'], 
               'auroc':['mean','std'], 
               'auprc':['mean','std']}
    # Calculate mean and standard dev
    df2 =  df.groupby(['methods','mode','evaluation']).agg(evaluation_d).reset_index()
    # Make a new column for convenience in planning
    df2["approach"] = df2["evaluation"] + " (" + df2["mode"] + ")"
    return df2

In [13]:
# Rename columns from tuples to simple strings for readability
columns = ["methods",'mode','evaluation', "balanced_acc.mean","balanced_acc.std","FDR.mean",
                    "FDR.std", "MCC.mean", "MCC.std", "F1.mean", "F1.std", "AUROC.mean",
                     "AUROC.std", "AUPRC.mean","AUPRC.std", "approach"]

<h3>SLI</h3>

In [14]:
sli_stats = get_mean_and_std(sli_results)

In [15]:
sli_stats.head()

methods   mode evaluation balanced_accuracy            \
                                                    mean       std   
0      DeepWalk CBOW   test       DANS          0.593863  0.035387   
1      DeepWalk CBOW   test        UNS          0.739648  0.137852   
2      DeepWalk CBOW  train       DANS          0.703653  0.011222   
3      DeepWalk CBOW  train        UNS          0.849059  0.105726   
4  DeepWalk SkipGram   test       DANS          0.630810  0.042103   

  false_discovery_rate           matthews_correlation_coefficient            \
                  mean       std                             mean       std   
0             0.401498  0.018883                         0.200131  0.072887   
1             0.176195  0.129827                         0.499108  0.266466   
2             0.316949  0.044127                         0.437531  0.044576   
3             0.118643  0.076121                         0.703968  0.205617   
4             0.368825  0.018729                         0.283764  0.093181   

   f1_score               auroc               auprc                approach  
       mean       std      mean       std      mean       std                
0  0.567944  0.114981  0.611760  0.031970  0.578986  0.019666   DANS (test)  
1  0.672132  0.197845  0.790910  0.169820  0.799302  0.148248    UNS (test)  
2  0.722168  0.046323  0.785701  0.035117  0.751405  0.033091  DANS (train)  
3  0.834441  0.121654  0.895227  0.094321  0.891749  0.075789   UNS (train)  
4  0.614877  0.117437  0.652207  0.077787  0.626466  0.036274   DANS (test)

In [16]:
# rename for better legibility
sli_stats.columns = columns
sli_stats.head()

,methods,mode,evaluation,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,0.593863,0.035387,0.401498,0.018883,0.200131,0.072887,0.567944,0.114981,0.611760,0.031970,0.578986,0.019666,DANS (test)
1,DeepWalk CBOW,test,UNS,0.739648,0.137852,0.176195,0.129827,0.499108,0.266466,0.672132,0.197845,0.790910,0.169820,0.799302,0.148248,UNS (test)
2,DeepWalk CBOW,train,DANS,0.703653,0.011222,0.316949,0.044127,0.437531,0.044576,0.722168,0.046323,0.785701,0.035117,0.751405,0.033091,DANS (train)
3,DeepWalk CBOW,train,UNS,0.849059,0.105726,0.118643,0.076121,0.703968,0.205617,0.834441,0.121654,0.895227,0.094321,0.891749,0.075789,UNS (train)
4,DeepWalk SkipGram,test,DANS,0.630810,0.042103,0.368825,0.018729,0.283764,0.093181,0.614877,0.117437,0.652207,0.077787,0.626466,0.036274,DANS (test)


In [17]:
sli_stats.to_csv("sli_stats.csv")

<h3>STRING</h3>

In [18]:
string_stats = get_mean_and_std(string_results)
string_stats.head()

methods   mode evaluation balanced_accuracy            \
                                                    mean       std   
0      DeepWalk CBOW   test       DANS          0.822263  0.016802   
1      DeepWalk CBOW   test        UNS          0.854046  0.022612   
2      DeepWalk CBOW  train       DANS          0.874667  0.034289   
3      DeepWalk CBOW  train        UNS          0.906246  0.039640   
4  DeepWalk SkipGram   test       DANS          0.866264  0.017862   

  false_discovery_rate           matthews_correlation_coefficient            \
                  mean       std                             mean       std   
0             0.149725  0.020909                         0.647233  0.032822   
1             0.087340  0.029481                         0.715841  0.044722   
2             0.135235  0.023526                         0.750840  0.069242   
3             0.078987  0.029631                         0.813801  0.078163   
4             0.111139  0.021461                         0.735877  0.030902   

   f1_score               auroc               auprc                approach  
       mean       std      mean       std      mean       std                
0  0.814773  0.019651  0.889122  0.011987  0.875925  0.018466   DANS (test)  
1  0.842817  0.025430  0.921659  0.020086  0.918986  0.012448    UNS (test)  
2  0.875555  0.036493  0.933964  0.021586  0.912928  0.012512  DANS (train)  
3  0.903771  0.042130  0.955888  0.024851  0.945469  0.011945   UNS (train)  
4  0.861634  0.024944  0.912387  0.012541  0.869357  0.022232   DANS (test)

In [19]:
# rename for better legibility
string_stats.columns = columns
string_stats.head()

,methods,mode,evaluation,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,0.822263,0.016802,0.149725,0.020909,0.647233,0.032822,0.814773,0.019651,0.889122,0.011987,0.875925,0.018466,DANS (test)
1,DeepWalk CBOW,test,UNS,0.854046,0.022612,0.087340,0.029481,0.715841,0.044722,0.842817,0.025430,0.921659,0.020086,0.918986,0.012448,UNS (test)
2,DeepWalk CBOW,train,DANS,0.874667,0.034289,0.135235,0.023526,0.750840,0.069242,0.875555,0.036493,0.933964,0.021586,0.912928,0.012512,DANS (train)
3,DeepWalk CBOW,train,UNS,0.906246,0.039640,0.078987,0.029631,0.813801,0.078163,0.903771,0.042130,0.955888,0.024851,0.945469,0.011945,UNS (train)
4,DeepWalk SkipGram,test,DANS,0.866264,0.017862,0.111139,0.021461,0.735877,0.030902,0.861634,0.024944,0.912387,0.012541,0.869357,0.022232,DANS (test)


In [20]:
string_stats.to_csv("string_stats.csv")